<a href="https://colab.research.google.com/github/gizdatalab/CPU/blob/main/classifier/CONDITIONALITY/CPU_CONDITIONAL_SentenceT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
! pip install datasets transformers sentencepiece huggingface_hub
! apt install git-lfs
! pip install sentence-transformers
! pip install optuna
! pip install evaluate
! pip install accelerate -U
! pip install codecarbon

In [ ]:
from codecarbon import EmissionsTracker

tracker = EmissionsTracker()
tracker.start()

[codecarbon INFO @ 17:31:19] [setup] RAM Tracking...
[codecarbon INFO @ 17:31:19] [setup] GPU Tracking...
[codecarbon INFO @ 17:31:19] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 17:31:19] [setup] CPU Tracking...
[codecarbon WARNING @ 17:31:19] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon WARNING @ 17:31:20] We saw that you have a AMD EPYC 7V12 64-Core Processor but we don't know it. Please contact us.
[codecarbon INFO @ 17:31:20] CPU Model on constant consumption mode: AMD EPYC 7V12 64-Core Processor
[codecarbon INFO @ 17:31:20] >>> Tracker's metadata:
[codecarbon INFO @ 17:31:20]   Platform system: Linux-5.15.0-1040-azure-x86_64-with-glibc2.10
[codecarbon INFO @ 17:31:20]   Python version: 3.8.5
[codecarbon INFO @ 17:31:20]   CodeCarbon version: 2.3.2
[codecarbon INFO @ 17:31:20]   Available RAM : 27.406 GB
[codecarbon INFO @ 17:31:20]   CPU count: 4
[codecarbon INFO @ 17:31:20]   CPU model: AMD EPYC 7V12 64-Core Processor
[codecarbon INFO @ 17:31:

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

[codecarbon INFO @ 17:31:35] Energy consumed for RAM : 0.000043 kWh. RAM Power : 10.2771635055542 W
[codecarbon INFO @ 17:31:35] Energy consumed for all GPUs : 0.000038 kWh. Total GPU Power : 9.098509586956894 W
[codecarbon INFO @ 17:31:35] Energy consumed for all CPUs : 0.000177 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 17:31:35] 0.000258 kWh of electricity used since the beginning.
No such comm: dc3b81c614ab453fac033b732cf3d302
[codecarbon INFO @ 17:33:20] Energy consumed for RAM : 0.000342 kWh. RAM Power : 10.2771635055542 W
[codecarbon INFO @ 17:33:20] Energy consumed for all GPUs : 0.000301 kWh. Total GPU Power : 9.036904437796405 W
[codecarbon INFO @ 17:33:20] Energy consumed for all CPUs : 0.001417 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 17:33:20] 0.002061 kWh of electricity used since the beginning.


In [ ]:
from datasets import load_dataset
import datasets
import numpy as np
import pandas as pd
import evaluate
import torch
import os
import sklearn.metrics as skm
from transformers import (AutoTokenizer, AutoConfig, AutoModelForSequenceClassification,
                          PreTrainedModel, BertModel, BertForSequenceClassification,
                          RobertaForSequenceClassification,
                          TrainingArguments, Trainer, TrainerCallback)
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

[codecarbon INFO @ 17:31:50] Energy consumed for RAM : 0.000086 kWh. RAM Power : 10.2771635055542 W
[codecarbon INFO @ 17:31:50] Energy consumed for all GPUs : 0.000076 kWh. Total GPU Power : 9.074139751588271 W
[codecarbon INFO @ 17:31:50] Energy consumed for all CPUs : 0.000354 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 17:31:50] 0.000516 kWh of electricity used since the beginning.
2024-01-22 17:31:52.578736: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-22 17:31:55.984160: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-01-22 17:31:55.9

In [ ]:
# Load Dataset from Hugging Face
try:
    dataset = load_dataset("GIZ/policy_classification", data_files="policy_classification.json",use_auth_token=True )
    data = pd.DataFrame(dataset['train'])
except:
    from huggingface_hub import hf_hub_download
    hf_hub_download("GIZ/policy_classification", filename="policy_classification.json", repo_type="dataset",use_auth_token=True,
    local_dir= 'data/')
    data = pd.read_json('data/policy_classification.json')


Using custom data configuration GIZ--policy_classification-cdc4b717b1bd720b


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Failed to read file '/home/azureuser/.cache/huggingface/datasets/downloads/dee24a3afc63cd87c4a0a9517165e43822030f22df905fa089a2628e8d786ca4' with error <class 'pyarrow.lib.ArrowInvalid'>: JSON parse error: Column() changed from object to array in row 0


In [ ]:
data['check'] = data.apply(lambda x: True if (x['UnconditionalLabel'] == True)
                          or (x['ConditionalLabel'] == True) else False,axis=1)
temp = data[data.check==True].reset_index(drop=True)
print(len(temp))
temp2 = data[data.check ==False].reset_index(drop=True).sample(frac= 0.3)
print(len(temp2))
data = pd.concat([temp,temp2], ignore_index=True)


3416
3177


[codecarbon INFO @ 17:34:50] Energy consumed for RAM : 0.000599 kWh. RAM Power : 10.2771635055542 W
[codecarbon INFO @ 17:34:50] Energy consumed for all GPUs : 0.000526 kWh. Total GPU Power : 9.00733048954448 W
[codecarbon INFO @ 17:34:50] Energy consumed for all CPUs : 0.002479 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 17:34:50] 0.003605 kWh of electricity used since the beginning.


# Manual Inputs


In [ ]:
###### Define Path to data ############################
repo_id = "GIZ/policy_classification"
filename="policy_classification.json"


### Define labels ##################
label_names= ['UnconditionalLabel','ConditionalLabel']


#######  Define Model params ######################

# define the model checkpoint
model_checkpoint = "climatebert/distilroberta-base-climate-f"

# define the problem type
problem_type="multi_label_classification"


######## Training Arguments ##################
output_dir = "CONDITIONAL-multilabel-climatebert_w"

# logging and evaluation
evaluation_strategy = "epoch"
logging_strategy="epoch"
logging_steps = 1
eval_steps = 1

# Important params
learning_rate=9.2e-6   #4.04e-5
batch_size = 8
num_train_epochs= 10
weight_decay=0.06
warmup_steps = 200
gradient_accumulation_steps = 2

[codecarbon INFO @ 17:37:05] Energy consumed for RAM : 0.000985 kWh. RAM Power : 10.2771635055542 W
[codecarbon INFO @ 17:37:05] Energy consumed for all GPUs : 0.000863 kWh. Total GPU Power : 9.064410723366866 W
[codecarbon INFO @ 17:37:05] Energy consumed for all CPUs : 0.004073 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 17:37:05] 0.005921 kWh of electricity used since the beginning.


# Automated

## Dataset

In [ ]:
# Load Dataset from Hugging Face
# Load Dataset from Hugging Face
# try:
#     dataset = load_dataset(repo_id, data_files=filename,
#                              use_auth_token=True )
#     data = pd.DataFrame(dataset['train'])
# except:
#     from huggingface_hub import hf_hub_download
#     hf_hub_download(repo_id, filename=filename,
#                     repo_type="dataset",use_auth_token=True,
#     local_dir= 'data/')
#     data = pd.read_json('data/policy_classification.json')

# the dataset already contains train_test split flag we use
# it to create train-test split.

train_df = data[data.split== 'Train'].reset_index(drop=True)
test_df = data[data.split=='Test'].reset_index(drop=True)

# Sort the labels and create plaecholders for id2label and label2id
label_names.sort()

num_labels = len(label_names)
id2label = {idx:label for idx, label in enumerate(label_names)}
label2id = {label:idx for idx, label in enumerate(label_names)}
print(id2label)

{0: 'ConditionalLabel', 1: 'UnconditionalLabel'}


In [ ]:
def prepare_multilabel_df(df, cols):
    """
    Takes the dataframe and list of columns which need to combined into
    One-hot-encoded labels.

    """
    df.dropna(subset = cols, inplace=True)
    df = df.reset_index(drop=True)
    df['labels'] = df.apply(lambda x: [int(x[label]) for label in cols],axis=1)
    # we have multilingual dataset ('en', 'fr','es'), in case of non-english the
    # translated Context is udner the column 'TranslatedContext'. We impute these
    # back into 'context' to work with only English dataset.
    df['Context'] = df.apply(lambda x: x['Context'] if x['Language'] == 'en'
                                    else x['TranslatedContext'][0],axis=1)
    df.rename(columns = {'Context':'text'}, inplace=True)

    df = df[['text','labels']+cols]
    return df

# creating multilabel dataset
train_df = prepare_multilabel_df(train_df, label_names)
test_df = prepare_multilabel_df(test_df, label_names)

print('\n','Classes Representation in Training Dataset:',len(train_df))
for i,label in enumerate(label_names):
    print(i,".",label, ":", sum( [x[i] for x in list(train_df['labels'])]))

print('\n','Classes Representation in Test Dataset:',len(test_df))
for i,label in enumerate(label_names):
    print(i,".",label, ":", sum( [x[i] for x in list(test_df['labels'])]))


 Classes Representation in Training Dataset: 5899
0 . ConditionalLabel : 1986
1 . UnconditionalLabel : 1312

 Classes Representation in Test Dataset: 694
0 . ConditionalLabel : 228
1 . UnconditionalLabel : 160


[codecarbon INFO @ 17:37:20] Energy consumed for RAM : 0.001027 kWh. RAM Power : 10.2771635055542 W
[codecarbon INFO @ 17:37:20] Energy consumed for all GPUs : 0.000901 kWh. Total GPU Power : 9.11275963113797 W
[codecarbon INFO @ 17:37:20] Energy consumed for all CPUs : 0.004250 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 17:37:20] 0.006179 kWh of electricity used since the beginning.


In [ ]:
for i,label in enumerate(label_names):
    # df_train[sector] = df_train.apply(lambda x: x['sector_label'][i], axis =1)
    print(i,".",label, ":", sum(train_df[label]))

# these are weights which can be used in building weighted loss if class is imbalanced
positive_weights = {}
negative_weights = {}
for label in label_names:
    positive_weights[label] = train_df.shape[0]/(2*np.count_nonzero(train_df[label]==True))
    negative_weights[label] = train_df.shape[0]/(2*np.count_nonzero(train_df[label]==False))
print("Positive Weights",'\n',positive_weights)
print("Negative Weights",'\n',negative_weights)

# as we dont want to miss anything (high recall) we use positive weights calculated above.
pos_weights = list(positive_weights.values())

# If using GPU we need to place all required data on else there will be error
posweights = torch.FloatTensor(pos_weights).to(device)

0 . ConditionalLabel : 1986
1 . UnconditionalLabel : 1312
Positive Weights 
 {'ConditionalLabel': 1.4851460221550856, 'UnconditionalLabel': 2.248094512195122}
Negative Weights 
 {'ConditionalLabel': 0.7537694863276259, 'UnconditionalLabel': 0.6430128624373229}


[codecarbon INFO @ 17:37:35] Energy consumed for RAM : 0.001070 kWh. RAM Power : 10.2771635055542 W
[codecarbon INFO @ 17:37:35] Energy consumed for all GPUs : 0.000996 kWh. Total GPU Power : 22.783951776260608 W
[codecarbon INFO @ 17:37:35] Energy consumed for all CPUs : 0.004427 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 17:37:35] 0.006494 kWh of electricity used since the beginning.


`pos_weight > 1 will increase the recall while pos_weight < 1 will increase the precision.`

## Model

In [ ]:
# problem_type, is not needed in tokenizer but keeping it for conformity
# https://huggingface.co/docs/transformers/main_classes/configuration?highlight=multi_label_classification#transformers.PretrainedConfig
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint,problem_type=problem_type)

# Set the model config and assign it and device to model
config = AutoConfig.from_pretrained(model_checkpoint, num_labels=num_labels, id2label=id2label, label2id=label2id, problem_type=problem_type)
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint,config=config).to(device)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at climatebert/distilroberta-base-climate-f and are newly initialized: ['classifier.out_proj.weight', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# we need datasets format to work with
train_ds = datasets.Dataset.from_pandas(train_df[['text','labels']])
test_ds =  datasets.Dataset.from_pandas(test_df[['text','labels']])
train_ds = train_ds.shuffle(seed=7)
test_ds = test_ds.shuffle(seed=7)


# Need to tokenize the data using the tokenizer of the model
def tokenize_and_encode(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True,
                        max_length=384)

cols = train_ds.column_names
cols.remove("labels")
print('Training data:',train_ds.num_rows)
print('Validation data:',test_ds.num_rows)

train_tokenized = train_ds.map(tokenize_and_encode, batched=True, remove_columns= cols)
val_tokenized = test_ds.map(tokenize_and_encode, batched=True, remove_columns= cols)

# need this to avoid error due to type mismatch
# https://discuss.pytorch.org/t/multi-label-binary-classification-result-type-float-cant-be-cast-to-the-desired-output-type-long/117915/3
train_tokenized.set_format("torch")
train_tokenized = (train_tokenized
          .map(lambda x : {"float_labels": x["labels"].to(torch.float)}, remove_columns=["labels"])
          .rename_column("float_labels", "labels"))

val_tokenized.set_format("torch")
val_tokenized = (val_tokenized
          .map(lambda x : {"float_labels": x["labels"].to(torch.float)}, remove_columns=["labels"])
          .rename_column("float_labels", "labels"))

Parameter 'function'=<function tokenize_and_encode at 0x7f9a922e01f0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


Training data: 5899
Validation data: 694


  0%|          | 0/6 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/5899 [00:00<?, ?ex/s]

[codecarbon INFO @ 17:37:50] Energy consumed for RAM : 0.001113 kWh. RAM Power : 10.2771635055542 W
[codecarbon INFO @ 17:37:50] Energy consumed for all GPUs : 0.001101 kWh. Total GPU Power : 25.062195979782075 W
[codecarbon INFO @ 17:37:50] Energy consumed for all CPUs : 0.004604 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 17:37:50] 0.006818 kWh of electricity used since the beginning.


  0%|          | 0/694 [00:00<?, ?ex/s]

## Metrics

In [ ]:
multilabel_f1_metric = evaluate.load("f1", "multilabel")
multilabel_recall_metric = evaluate.load("recall","multilabel")
multilabel_precision_metric = evaluate.load("precision","multilabel")

def get_scores(y_pred, y_test, thresh=0.5, sigmoid=True):
    """
    Takes the Predictions and Truth Values, and return multilabel metrics
    aggregated on multiple strategies['micro', 'sample', 'weighted']

    """
    # we get raw function value, therefore we apply sigmoid to get to the
    # probabilities
    if problem_type == "multi_label_classification":
        y_pred = torch.from_numpy(y_pred)
        y_test = torch.from_numpy(y_test)
        if sigmoid:
          y_pred = y_pred.sigmoid()
          y_pred = (y_pred > thresh)

        return {
            "precision-micro": multilabel_precision_metric.compute(predictions=y_pred, references=y_test, average="micro")["precision"],
            "precision-samples": multilabel_precision_metric.compute(predictions=y_pred, references=y_test, average="samples")["precision"],
            "precision-weighted": multilabel_precision_metric.compute(predictions=y_pred, references=y_test, average="weighted")["precision"],
            "recall-micro": multilabel_recall_metric.compute(predictions=y_pred, references=y_test, average="micro")["recall"],
            "recall-samples": multilabel_recall_metric.compute(predictions=y_pred, references=y_test, average="samples")["recall"],
            "recall-weighted": multilabel_recall_metric.compute(predictions=y_pred, references=y_test, average="weighted")["recall"],
            "f1-micro": multilabel_f1_metric.compute(predictions=y_pred, references=y_test, average="micro")["f1"],
            "f1-samples": multilabel_f1_metric.compute(predictions=y_pred, references=y_test, average="samples")["f1"],
            "f1-weighted": multilabel_f1_metric.compute(predictions=y_pred, references=y_test, average="weighted")["f1"],
            }
    else:
        y_pred = torch.from_numpy(y_pred).argmax(dim=1)
        y_test = torch.from_numpy(y_test)

        report = skm.classification_report(y_test, y_pred, output_dict=True)
        df_report = pd.DataFrame(report).transpose()
        return {"Precision_macro": df_report.loc['macro avg']['precision'],
                "Precision_weighted": df_report.loc['weighted avg']['precision'],
                "Recall_macro": df_report.loc['macro avg']['recall'],
                "Recall_weighted": df_report.loc['weighted avg']['recall'],
                "F1-Score": df_report.loc['macro avg']['f1-score'],
                "Accuracy": skm.accuracy_score(y_test, y_pred)}


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    return get_scores(predictions, labels)

## Training arguments

In [ ]:
args = TrainingArguments(
    output_dir,
    logging_strategy=logging_strategy,
    evaluation_strategy = evaluation_strategy,
    save_strategy = "epoch",
    logging_steps = logging_steps,
    eval_steps= eval_steps,
    learning_rate=learning_rate,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_train_epochs,
    weight_decay=weight_decay,
    lr_scheduler_type = "linear",
    gradient_accumulation_steps = gradient_accumulation_steps,
    warmup_steps = warmup_steps,
    push_to_hub = True,
)

## Custom Trainer

In [ ]:
# for class weights we need to use Custom Multi-Label Trainer
# In multi-label problem we will be using Binary Cross Entropy loss with
# sigmoid layer on top rather than softmax.
class MultilabelTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits
        loss_fct = torch.nn.BCEWithLogitsLoss(pos_weight=posweights)
        loss = loss_fct(logits.view(-1, self.model.config.num_labels),
                        labels.float().view(-1, self.model.config.num_labels))
        return (loss, outputs) if return_outputs else loss


# Training

In [ ]:
multi_trainer =  MultilabelTrainer(
    model,
    args,
    train_dataset=train_tokenized,
    eval_dataset=val_tokenized,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer)


[codecarbon INFO @ 17:38:05] [setup] RAM Tracking...
[codecarbon INFO @ 17:38:05] [setup] GPU Tracking...
[codecarbon INFO @ 17:38:05] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 17:38:05] [setup] CPU Tracking...
[codecarbon WARNING @ 17:38:05] No CPU tracking mode found. Falling back on CPU constant mode.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
[codecarbon INFO @ 17:38:05] Energy consumed for RAM : 0.001156 kWh. RAM Power : 10.2771635055542 W
[codecarbon INFO @ 17:38:05] Energy consumed for all GPUs : 0.001205 kWh. Total GPU Power : 25.0062146960673 W
[codecarbon INFO @ 17:38:05] Energy consumed for all CPUs : 0.004781 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 17:38:05] 0.007142 kWh of electri

In [ ]:
multi_trainer.train()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
You're using a RobertaTokeni

Epoch,Training Loss,Validation Loss,Precision-micro,Precision-samples,Precision-weighted,Recall-micro,Recall-samples,Recall-weighted,F1-micro,F1-samples,F1-weighted
1,0.797800,0.727394,0.462415,0.230548,0.475892,0.523196,0.262968,0.523196,0.490931,0.239193,0.491374
2,0.683100,0.633300,0.547862,0.328530,0.553368,0.693299,0.355908,0.693299,0.612059,0.334774,0.613604
3,0.585200,0.585721,0.620455,0.346542,0.620958,0.703608,0.361671,0.703608,0.659420,0.348703,0.659389
4,0.498500,0.577788,0.587045,0.362392,0.593674,0.747423,0.386888,0.747423,0.657596,0.367435,0.658818
5,0.434600,0.558716,0.641425,0.374640,0.641590,0.742268,0.384726,0.742268,0.688172,0.374640,0.688173
6,0.370600,0.563554,0.652466,0.388329,0.653411,0.750000,0.391931,0.750000,0.697842,0.385207,0.698022
7,0.321100,0.570808,0.678218,0.370317,0.678590,0.706186,0.368876,0.706186,0.691919,0.365514,0.691876
8,0.287300,0.584560,0.655844,0.398415,0.663975,0.780928,0.405620,0.780928,0.712941,0.397695,0.713574
9,0.255200,0.584484,0.653105,0.407061,0.656167,0.786082,0.410663,0.786082,0.713450,0.404419,0.713918
10,0.238300,0.586391,0.669704,0.394813,0.672051,0.757732,0.395533,0.757732,0.711004,0.390970,0.711260


[codecarbon INFO @ 17:38:20] Energy consumed for RAM : 0.001199 kWh. RAM Power : 10.2771635055542 W
[codecarbon INFO @ 17:38:20] Energy consumed for all GPUs : 0.001384 kWh. Total GPU Power : 42.93735631758471 W
[codecarbon INFO @ 17:38:20] Energy consumed for all CPUs : 0.004958 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 17:38:20] 0.007541 kWh of electricity used since the beginning.
[codecarbon INFO @ 17:38:27] Energy consumed for RAM : 0.000043 kWh. RAM Power : 10.2771635055542 W
[codecarbon INFO @ 17:38:27] Energy consumed for all GPUs : 0.000255 kWh. Total GPU Power : 61.271385635493665 W
[codecarbon INFO @ 17:38:27] Energy consumed for all CPUs : 0.000177 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 17:38:27] 0.000475 kWh of electricity used since the beginning.
[codecarbon INFO @ 17:38:35] Energy consumed for RAM : 0.001241 kWh. RAM Power : 10.2771635055542 W
[codecarbon INFO @ 17:38:35] Energy consumed for all GPUs : 0.001671 kWh. Total GPU Power : 68.90387689667952 W

TrainOutput(global_step=3690, training_loss=0.44718566108848345, metrics={'train_runtime': 2286.5802, 'train_samples_per_second': 25.798, 'train_steps_per_second': 1.614, 'total_flos': 5860688885038080.0, 'train_loss': 0.44718566108848345, 'epoch': 10.0})

[codecarbon INFO @ 18:16:51] Energy consumed for RAM : 0.007791 kWh. RAM Power : 10.2771635055542 W
[codecarbon INFO @ 18:16:51] Energy consumed for all GPUs : 0.043923 kWh. Total GPU Power : 27.79972841918477 W
[codecarbon INFO @ 18:16:51] Energy consumed for all CPUs : 0.032230 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 18:16:51] 0.083944 kWh of electricity used since the beginning.
[codecarbon INFO @ 18:17:06] Energy consumed for RAM : 0.007834 kWh. RAM Power : 10.2771635055542 W
[codecarbon INFO @ 18:17:06] Energy consumed for all GPUs : 0.044036 kWh. Total GPU Power : 27.332389948867586 W
[codecarbon INFO @ 18:17:06] Energy consumed for all CPUs : 0.032407 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 18:17:06] 0.084278 kWh of electricity used since the beginning.


In [ ]:
tracker.stop()

[codecarbon INFO @ 18:17:11] Energy consumed for RAM : 0.007849 kWh. RAM Power : 10.2771635055542 W
[codecarbon INFO @ 18:17:11] Energy consumed for all GPUs : 0.044076 kWh. Total GPU Power : 27.470470501131715 W
[codecarbon INFO @ 18:17:11] Energy consumed for all CPUs : 0.032469 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 18:17:11] 0.084393 kWh of electricity used since the beginning.


0.030843792505634335

In [ ]:
multi_trainer.push_to_hub()

'https://huggingface.co/ppsingh/CONDITIONAL-multilabel-climatebert_w/tree/main/'

In [ ]:
# class weight
log_results = multi_trainer.state.log_history
log_results

[{'loss': 0.7978,
  'learning_rate': 8.754498567335243e-06,
  'epoch': 1.0,
  'step': 369},
 {'eval_loss': 0.7273943424224854,
  'eval_precision-micro': 0.4624145785876993,
  'eval_precision-samples': 0.23054755043227665,
  'eval_precision-weighted': 0.4758915043551704,
  'eval_recall-micro': 0.5231958762886598,
  'eval_recall-samples': 0.26296829971181557,
  'eval_recall-weighted': 0.5231958762886598,
  'eval_f1-micro': 0.4909310761789601,
  'eval_f1-samples': 0.23919308357348704,
  'eval_f1-weighted': 0.4913740874189967,
  'eval_runtime': 7.945,
  'eval_samples_per_second': 87.351,
  'eval_steps_per_second': 10.95,
  'epoch': 1.0,
  'step': 369},
 {'loss': 0.6831,
  'learning_rate': 7.781776504297994e-06,
  'epoch': 2.0,
  'step': 738},
 {'eval_loss': 0.6333003044128418,
  'eval_precision-micro': 0.5478615071283096,
  'eval_precision-samples': 0.3285302593659942,
  'eval_precision-weighted': 0.5533676460738703,
  'eval_recall-micro': 0.6932989690721649,
  'eval_recall-samples': 0.355

In [ ]:
import json
with open("classifier_info/conditional_climatebert_logs_w.json", "w") as fp:
    json.dump(log_results , fp)

# Predict and Evaluate

## Trainer

In [ ]:
predictions= multi_trainer.predict(val_tokenized)
pred,labels,_ = predictions
y_pred = torch.from_numpy(pred)
y_true = torch.from_numpy(labels)
y_prob = y_pred.sigmoid()
thresh = 0.5
y_pred = (y_prob>=thresh).bool()

/anaconda/envs/azureml_py38/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/anaconda/envs/azureml_py38/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/anaconda/envs/azureml_py38/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1514: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(


In [ ]:
# class weights
cm = skm.multilabel_confusion_matrix(y_true, y_pred)
for i,label in enumerate(label_names):
    print(label)
    print(cm[i],'\n')

ConditionalLabel
[[393  73]
 [ 60 168]] 

UnconditionalLabel
[[462  72]
 [ 34 126]] 



In [ ]:
# classw eights
report = skm.classification_report(y_true, y_pred, output_dict=True)
df_report = pd.DataFrame(report).transpose()
df_report = df_report.reset_index()
mapping = {str(i):j for i,j in enumerate(label_names)}
df_report['index'] = df_report['index'].map(mapping).fillna(df_report['index'])
df_report.rename(columns = {'index':'label'}, inplace=True)
df_report

/anaconda/envs/azureml_py38/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/anaconda/envs/azureml_py38/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,label,precision,recall,f1-score,support
0,ConditionalLabel,0.697095,0.736842,0.716418,228.0
1,UnconditionalLabel,0.636364,0.787500,0.703911,160.0
2,micro avg,0.669704,0.757732,0.711004,388.0
3,macro avg,0.666730,0.762171,0.710164,388.0
4,weighted avg,0.672051,0.757732,0.711260,388.0
5,samples avg,0.394813,0.395533,0.390970,388.0


In [ ]:
report = skm.classification_report(y_true, y_pred, output_dict=True)
df_report = pd.DataFrame(report).transpose()
df_report = df_report.reset_index()
mapping = {str(i):j for i,j in enumerate(label_names)}
df_report['index'] = df_report['index'].map(mapping).fillna(df_report['index'])
df_report.rename(columns = {'index':'label'}, inplace=True)
df_report

/anaconda/envs/azureml_py38/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,label,precision,recall,f1-score,support
0,Agriculture,0.818930,0.843220,0.830898,236.0
1,Buildings,0.647059,0.523810,0.578947,21.0
2,Coastal Zone,0.757143,0.679487,0.716216,78.0
3,Cross-Cutting Area,0.708108,0.642157,0.673522,204.0
4,Disaster Risk Management (DRM),0.662651,0.597826,0.628571,92.0
5,Economy-wide,0.530864,0.430000,0.475138,100.0
6,Education,0.812500,0.481481,0.604651,27.0
7,Energy,0.862179,0.853968,0.858054,315.0
8,Environment,0.652174,0.631579,0.641711,95.0
9,Health,0.838235,0.750000,0.791667,76.0


## Pipeline

In [ ]:
from transformers import pipeline
model_checkpoint = "ppsingh/TAPP-multilabel-climatebert"
pipe = pipeline("text-classification", model=model_checkpoint, top_k=None, device=device)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
predictions = pipe(list(test_df['text']))
pred = []
for prediction in predictions:
    pred.append({x['label']:x['score'] for x in prediction})
df_pred = pd.DataFrame(pred)
df_pred['labels'] = df_pred.apply(lambda x: np.array([x[label]
                            for label in label_names]) >= 0.50,axis=1)
y_true = np.array(list(test_df['labels']))
y_pred = np.array(list(df_pred['labels']))

In [ ]:
cm = skm.multilabel_confusion_matrix(y_true, y_pred)
for i,label in enumerate(label_names):
    print(label)
    print(cm[i],'\n')

ActionLabel
[[417 104]
 [ 69 531]] 

PlansLabel
[[807  77]
 [ 71 166]] 

PolicyLabel
[[939  30]
 [ 28 124]] 

TargetLabel
[[737  62]
 [ 37 285]] 



In [ ]:
report = skm.classification_report(y_true, y_pred, output_dict=True)
df_report = pd.DataFrame(report).transpose()
df_report = df_report.reset_index()
mapping = {str(i):j for i,j in enumerate(label_names)}
df_report['index'] = df_report['index'].map(mapping).fillna(df_report['index'])
df_report.rename(columns = {'index':'label'}, inplace=True)
df_report

/anaconda/envs/azureml_py38/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/anaconda/envs/azureml_py38/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,label,precision,recall,f1-score,support
0,ActionLabel,0.836220,0.885000,0.859919,600.0
1,PlansLabel,0.683128,0.700422,0.691667,237.0
2,PolicyLabel,0.805195,0.815789,0.810458,152.0
3,TargetLabel,0.821326,0.885093,0.852018,322.0
4,micro avg,0.802030,0.843631,0.822305,1311.0
5,macro avg,0.786467,0.821576,0.803515,1311.0
6,weighted avg,0.801289,0.843631,0.821827,1311.0
7,samples avg,0.787021,0.808802,0.782698,1311.0
